In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import linear_model   # sklearn으로 logistic 구현
from sklearn.preprocessing import MinMaxScaler   # 정규화 진행
from scipy import stats   # 이상치 처리
import matplotlib.pyplot as plt


# 경고메시지 출력하지 않음!
warnings.filterwarnings(action='ignore')

# Raw Data Loading
df = pd.read_csv('../data/admission.csv')

#############################
# preprocessing
#############################

# 결측치부터 살펴봐야 한다!
# print(df.isnull().sum())   # 결치값은 없다!
# df.info()

# 이상치가 있는지를 확인하고 처리해보자!
# 종속변수의 이상치를 outlier
# 독립변수의 이상치를 지대값
# 이상치가 존재하는지를 눈으로 확인하는 가장 쉬운 방법은 boxplot
# figure = plt.figure()
# ax1 = figure.add_subplot(1,4,1)
# ax2 = figure.add_subplot(1,4,2)
# ax3 = figure.add_subplot(1,4,3)
# ax4 = figure.add_subplot(1,4,4)
# ax1.set_title('ADMIT')
# ax2.set_title('GRE')
# ax3.set_title('GPA')
# ax4.set_title('RANK')

# ax1.boxplot(df['admit'])
# ax2.boxplot(df['gre'])
# ax3.boxplot(df['gpa'])
# ax4.boxplot(df['rank'])

# figure.tight_layout()
# plt.show()
# boxplot을 이용해서 눈으로 확인해보니 이상치가 존재한다!
# z-score를 이용해서 이상치를 제거하고 진행해보자!

zscore_threshold = 2.0

for col in df.columns:
    outlier = df[col][np.abs(stats.zscore(df[col])) > zscore_threshold]
    df = df.loc[~df[col].isin(outlier)]
    
# 이상치를 제거했으니 정규화를 진행
# display(df.head())
x_data = df.drop('admit', axis=1, inplace=False)
t_data = df['admit'].values.reshape(-1,1)
# t_data는 0과 1로만 구성되어 있다. 따라서 정규화를 할 필요가 없다.

# 정규화를 하기 위해 scaler
scaler = MinMaxScaler()
scaler.fit(x_data)

norm_x_data = scaler.transform(x_data)
# print(norm_x_data)

# training data set
# norm_x_data
# t_data

### sklearn 구현

model = linear_model.LogisticRegression()

model.fit(x_data, t_data)

my_score = np.array([[600, 3.8, 1]])
predict_val = model.predict(my_score)           # 0 or 1로 결과 도출
predict_proba = model.predict_proba(my_score)   # 확률값으로 결과를 도출

print('sklearn의 결과 : 합격여부 : {}, 확률 : {}'.format(predict_val,
                                                         predict_proba))

###############################
# tensorflow 구현

# training data set
# norm_x_data
# t_data

# placeholder
X = tf.placeholder(shape=[None,3], dtype=tf.float32)
T = tf.placeholder(shape=[None,1], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random.normal([3,1]))
b = tf.Variable(tf.random.normal([1]))

# Hypothesis, model, predict model, logistic regression model
logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

# loss function, cross entropy, log loss라고 하기도 한다.
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit,
                                                              labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-4).minimize(loss)

# Session, 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 반복학습
for step in range(300000):
    _, loss_val = sess.run([train, loss],
                           feed_dict={X: norm_x_data,
                                      T: t_data})
    
    if step % 30000 == 0:
        print('loss의 값 : {}'.format(loss_val))
        

C:\Users\shcho\anaconda3\envs\machine_TF15\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


sklearn의 결과 : 합격여부 : [1], 확률 : [[0.43740782 0.56259218]]



C:\Users\shcho\anaconda3\envs\machine_TF15\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
C:\Users\shcho\anaconda3\envs\machine_TF15\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



loss의 값 : 1.3669615983963013
loss의 값 : 0.6717689633369446
loss의 값 : 0.6183386445045471
loss의 값 : 0.611027181148529
loss의 값 : 0.6072802543640137
loss의 값 : 0.6041977405548096
loss의 값 : 0.6015024185180664
loss의 값 : 0.5991346836090088
loss의 값 : 0.5970442891120911
loss의 값 : 0.5951951742172241


In [27]:
# predict
my_score = np.array([[600, 3.8, 1]])
norm_my_score = scaler.transform(my_score)

result = sess.run(H, feed_dict={X: norm_my_score})
print('tensorflow로 예측한 결과 : {}'.format(result))   # [[0.45585647]]
# tensorflow로 예측한 결과는 탈락!


tensorflow로 예측한 결과 : [[0.45585647]]


C:\Users\shcho\anaconda3\envs\machine_TF15\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  "X does not have valid feature names, but"


In [35]:
### Regression의 Metrics를 알아보자!

### 예제는 Ozone을 이용해서 Model을 만들어보자!
### model은 sklearn으로 구현해보자!

import numpy as np
import pandas as pd
from sklearn import linear_model
from scipy import stats
from sklearn.model_selection import train_test_split
# 데이터셋을 train, test(validation)로 나눠준다.


### Raw Data Loading
df = pd.read_csv('../data/ozone.csv')
# print(df.shape)   # (153, 6)

# 결측치부터 제거
training_data = df.dropna(how='any', inplace=False)
# print(training_data.shape)   # (111, 6)

# 이상치도 제거
zscore_threshold = 2.0

for col in training_data.columns:
    outlier = training_data[col][np.abs(stats.zscore(training_data[col])) > zscore_threshold]
    training_data = training_data.loc[~training_data[col].isin(outlier)]
    
# sklearn으로 구현할거라서 정규화 처리는 하지 않는다!
# display(training_data.head())

# Data Set
x_data = training_data[['Solar.R', 'Wind', 'Temp']].values
t_data = training_data['Ozone'].values.reshape(-1,1)

# Train / Validation Data Set
train_x_data, valid_x_data, train_t_data, valid_t_data = \
train_test_split(x_data,
                 t_data,
                 test_size=0.3,
                 random_state=2)   # random의 seed 역할

# Model
model = linear_model.LinearRegression()

# Model 학습
model.fit(train_x_data, train_t_data)

# 예측값(predict_value)
# 정답(valid_t_data)
predict_value = model.predict(valid_x_data)

# 예측값과 정답간의 차이가 작으면 작을수록 좋은 것!

In [36]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(valid_t_data, predict_value))   # 13.924465776324642

13.924465776324642


In [37]:
from sklearn.metrics import mean_squared_error

print(mean_squared_error(valid_t_data, predict_value))   # 271.5671192367061

271.5671192367061


In [38]:
from sklearn.metrics import r2_score

print(r2_score(valid_t_data, predict_value))   # 0.3734728354920861

0.3734728354920861
